# Taller 4

## Métodos Computacionales 2

Instrucciones: Suba el jupyter notebook **.ipynb** en la actividad de bloque neon. El archivo debe ser nombrado ApellidoNombreApellidoNombre_Taller4.ipynb, con su primer Apellido y Nombre. **El archivo debe poder correr en el binder del curso o en google colab**

Si un ejercicio demora **más de 10 segundos** en correr se considerará **incorrecto**.

La simulación no debería demorar más de **60 segundos** en el github del curso o en google colab.

No modifique el cuaderno, ni los modos de lectura de los archivos.

**Entrega: Sábado 30 de Septiembre 11:59 pm**

## Librerias

In [1]:
import time
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import animation
plt.style.use('dark_background')

In [2]:
from matplotlib import rc
rc("animation", html = "jshtml")

## Problema 2 (3/5) Puntos

Solucionar la ecuación de onda bidimensional con extremos fijos en cero la cual se encuentra entre dos medios cuyas velocidades de propagación son $c_1$ y $c_2$ para la posición inicial $p(x,y)$,

$$p(x,y) = \frac{1}{\sqrt{\sigma}}e^\left(-{\frac{(x-0.25)^2+(y-0.5)^2}{\sigma^2}}\right) + \frac{1}{\sqrt{\sigma}}e^\left(-{\frac{(x-0.75)^2+(y-0.5)^2}{\sigma^2}}\right)$$
$$p(x, y) = \frac{1}{\sqrt{\sigma}} e^{-\frac{(x-0.25)^2+(y-0.5)^2}{\sigma^2}} + \frac{1}{\sqrt{\sigma}} e^{-\frac{(x-0.75)^2+(y-0.5)^2}{\sigma^2}}$$

con $\sigma = 0.05$, en el dominio $x \in [0, 1]$,  $y \in [0, 1]$ y velocidad inicial de cada nodo igual a cero para $t=0$, la frontera entre los dos medios se encuentra en la posición $x = 0.5$. Use los valores de la tabla. Realice una animación 3D, para que la simulación no demore tanto realice la simulación con solo las imagenes de cada $50*dt$. Use el siguiente [notebook](https://github.com/diegour1/MetodosComputacionales2/blob/main/Notebooks/11%20-%20ecuacion_onda_1D.ipynb) como guía.

**Tabla:**

|  |                                        |
|-----------------------|----------------------------------------|
| $$c_1$$                 |                                $$1.5$$ |
| $$c_2$$                 |                                $$2.5$$ |
| $$dx$$                |                               $$0.05$$ |
| $$dy$$                |                               $$0.05$$ |
| $$t_{total}$$           |                                  $$0.5$$ |
| $$dt$$                | $$\frac{1}{10c_2}\frac{dx^2dy^2}{dx^2+dy^2}$$ |

En la calificación el notebook se correra en Google Colab, de cualquier forma adjunte la animación en un video de youtube público, debe obtener un video como el que se muestra,

Cambie este link por su video de youtube,

Espacio para el video de youtube

https://www.youtube.com/watch?v=heeGCgfhheY

In [13]:
## Su codigo aqui
#deltas
dx = 0.05
dy = 0.05
#fronteras
Lx = 1.0
Ly = 1.0
l0 = 0.0
lL = 0.0
t_max = 0.5
#medios
c1 = 1.5
c2 = 2.5
px = 0.5
#P(x,y) inicial
def pos_ini(x, y):
    sig = 0.05
    term1 = (1.0 / np.sqrt(sig)) * np.exp(-(((x - 0.25)**2) + ((y - 0.5)**2)) / (sig**2))
    term2 = (1.0 / np.sqrt(sig)) * np.exp(-(((x - 0.75)**2) + ((y - 0.5)**2)) / (sig**2))
    return term1 + term2
#Velocidad inicial
def vel_ini(x, y):
    return np.zeros_like(x)
#Solucion
def solucion_ecuacion_onda_2D(dx, dy, t_max, Lx, Ly, c1, c2, px, l0, lL):
    dt = (1/(10*c2)) * (((dx**2) * (dy**2))/((dx**2) + (dy**2)))

    r1x = (c1 * dt) / (dx**2)
    r2x = (c2 * dt) / (dx**2)
    r1y = (c1 * dt) / (dy**2)
    r2y = (c2 * dt) / (dy**2)

    a1 = 1 - 2*r1x - 2*r1y
    a2 = 1 - 2*r2x - 2*r2y

    Nx = int(np.round((Lx)/dx)) + 1 
    Ny = int(np.round((Ly)/dy)) + 1
    lt = int(np.round(t_max / dt)) + 1 

    x = np.linspace(0, Lx, Nx)
    y = np.linspace(0, Ly, Ny)
    t = np.linspace(0, t_max, lt)

    U = np.zeros((lt, Nx, Ny))
    X1, Y1 = np.meshgrid(x, y)
    U[0, :, :] = pos_ini(X1, Y1)

    X, Y = np.meshgrid(x, y, indexing='ij')
    U[:, 0, :] = 0
    U[:, -1, :] = 0
    v0 = vel_ini(X, Y)

    for j in range(1, Ny-1):
        for i in range(1, Nx - 1):
            if i < px * Nx:
                U[1, i, j] = (0.5 * a1 * U[0, i, j]) + 0.5 * r1x * (U[0, i - 1, j] + U[0, i + 1, j] + U[0, i, j - 1] + U[0, i, j + 1]) + (dt * v0[i, j])
            else:
                U[1, i, j] = (0.5 * a2 * U[0, i, j]) + 0.5 * r2x * (U[0, i - 1, j] + U[0, i + 1, j] + U[0, i, j - 1] + U[0, i, j + 1]) + (dt * v0[i, j])

    for n in range(1, lt-1):
        for i in range(1, Nx-1):
            for j in range(1, Ny-1):
                if i < px * Nx:
                    px = r1x*(U[n,i-1,j] + U[n,i+1,j])
                    py = r1y*(U[n,i,j-1] + U[n,i,j+1])
                    U[n+1,i,j] = (a1*U[n,i,j]) + px + py - U[n-1,i,j]
                else:
                    px = r2x*(U[n,i-1,j] + U[n,i+1,j])
                    py = r2y*(U[n,i,j-1] + U[n,i,j+1])
                    U[n+1,i,j] = (a2*U[n,i,j]) + px + py - U[n-1,i,j]
    X, Y = np.meshgrid(x, y)
    return X, Y, U
start_time = time.time()
X, Y, U = solucion_ecuacion_onda_2D(dx, dy, t_max, Lx, Ly, c1, c2, px, l0, lL)
print("Tiempo usado en calcular la solución (s):", time.time() - start_time)
print('x.shape:', X.shape, 'y.shape:', Y.shape, 'U.shape:', U.shape)
U = U[::25]
print('U.shape (subsampled):', U.shape)


Tiempo usado en calcular la solución (s): 11.22339415550232
x.shape: (21, 21) y.shape: (21, 21) U.shape: (10001, 21, 21)
U.shape (subsampled): (401, 21, 21)


In [ ]:
def update(frame_num, X, Y, U, ax):
  ax.clear()
  ax.set_zlim([np.min(U),np.max(U)])
  ax.view_init(elev=30, azim=-45 + frame_num * 2)
  colors = np.where(X < 0.5, 'blue', 'red')
  ax.plot_surface(X,Y,U[frame_num,:,:], facecolors=colors)

def crear_animacion_3D(X,Y,U, name):
  N = U.shape[0]
  fig = plt.figure(figsize=(9,6), dpi = 75)
  fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
  ax = fig.add_subplot(projection='3d')
  ax.xaxis.pane.fill = False
  ax.yaxis.pane.fill = False
  ax.zaxis.pane.fill = False
  ani = animation.FuncAnimation(fig, update, N, fargs=(X, Y, U, ax))

  return ani
  #ani.save('ecuacion_calor_2D.gif',fps=24)
  

start_time = time.time()
crear_animacion_3D(X, Y, U, 'cambio_de_medio_3D.gif')
#print("Tiempo usado en crear la animación (s):", time.time() - start_time)
